In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
280,AFG,Asia,Afghanistan,2020-10-06,39486.0,145.0,35.286,1467.0,5.0,1.571,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
492,ALB,Europe,Albania,2020-10-06,14410.0,144.0,145.571,400.0,4.0,2.857,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
773,DZA,Africa,Algeria,2020-10-06,52270.0,134.0,151.000,1768.0,8.0,7.000,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-10-05,France,634736.0,0.0,32281.0,0.0,0.0,0.0
0,2020-10-05,China,90651.0,0.0,4744.0,0.0,0.0,0.0
0,2020-10-05,Italy,327687.0,0.0,36026.0,0.0,0.0,0.0
0,2020-10-05,Spain,796998.0,0.0,32094.0,0.0,0.0,0.0
0,2020-10-05,United States,7497023.0,0.0,210983.0,0.0,0.0,0.0
0,2020-10-05,World,35815878.0,0.0,1048088.0,0.0,0.0,0.0
0,2020-10-05,United Kingdom,514173.0,0.0,42417.0,0.0,0.0,0.0
0,2020-10-05,Germany,302632.0,0.0,9541.0,0.0,0.0,0.0
0,2020-10-05,Iran,474991.0,0.0,27172.0,0.0,0.0,0.0
0,2020-10-05,Turkey,325641.0,0.0,8490.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.at[0,"total_cases_predict"]
    rez2 = res_tempo.at[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.at[0,"date"] = v0
    follow_df.at[0,"country"] = country
    follow_df.at[0,"total_cases_predict"] = rez1
    follow_df.at[0,"total_cases_real"] = v1
    follow_df.at[0,"total_deaths_predict"] = rez2
    follow_df.at[0,"total_deaths_real"] = v2
    follow_df.at[0,"error_abs_cases"] = errorCase1
    follow_df.at[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-10-06,France,634736.0,624274.0,32281.0,32299.0,10462.0,-18.0
0,2020-10-06,China,90651.0,90652.0,4744.0,4739.0,-1.0,5.0
0,2020-10-06,Italy,327687.0,327586.0,36026.0,36002.0,101.0,24.0
0,2020-10-05,Spain,796998.0,813412.0,32094.0,32225.0,-16414.0,-131.0
0,2020-10-06,United States,7497023.0,7458550.0,210983.0,210192.0,38473.0,791.0
0,2020-10-06,World,35815878.0,35523518.0,1048088.0,1042398.0,292360.0,5690.0
0,2020-10-06,United Kingdom,514173.0,515571.0,42417.0,42369.0,-1398.0,48.0
0,2020-10-06,Germany,302632.0,303258.0,9541.0,9546.0,-626.0,-5.0
0,2020-10-06,Iran,474991.0,475674.0,27172.0,27192.0,-683.0,-20.0
0,2020-10-06,Turkey,325641.0,326046.0,8490.0,8498.0,-405.0,-8.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")